In [1]:
import numpy as np
import string

In [3]:
np.random.seed(1234)

In [7]:
initial = {} # start of a phrase
first_order = {} # second word only
second_order = {} # stores 2nd-order transitions: (word1, word2) -> next word

In [8]:
def remove_punctuation(s):
    return s.translate(str.maketrans('','', string.punctuation))

In [9]:
with open(r"D:\Deepam\Projects\Text Classifier\robert_frost.txt", "r", encoding="utf-8") as f:
    frost_text = f.read()

print("\n=== Robert Frost ===")
print(frost_text[:500])


=== Robert Frost ===
Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there
Had worn them really about the same,

And both that morning equally lay
In leaves no step had trodden black.
Oh, I kept the first for another day! 
Yet knowing how way 


In [10]:
def add2dict(d, k, v):
    if k not in d:
        d[k] = []
    d[k].append(v)

# [cat, cat, dog, dog, dog, dog, dog, mouse, ...]

🔸 Purpose:
This helper function helps us build a dictionary of lists.

Each key k points to a list of values v.

🔸 How it works:
Let’s say we want to track the animals that come after each animal in a story. Something like this:
```python
cat cat dog dog dog dog dog mouse
```
We’ll use add2dict like this:
```python
d = {}
animals = ['cat', 'cat', 'dog', 'dog', 'dog', 'dog', 'dog', 'mouse']
for i in range(1, len(animals)):
    prev = animals[i - 1]
    curr = animals[i]
    add2dict(d, prev, curr)
    
Now what happens:

i	prev	curr	dict after add2dict
1	cat	cat	{'cat': ['cat']}
2	cat	dog	{'cat': ['cat', 'dog']}
3	dog	dog	{'cat': [...], 'dog': ['dog']}
4	dog	dog	{'dog': ['dog', 'dog']}
5	dog	dog	{'dog': ['dog', 'dog', 'dog']}
6	dog	dog	{'dog': ['dog', 'dog', 'dog', 'dog']}
7	dog	mouse	{'dog': [..., 'mouse']}

Final d:
{
  'cat': ['cat', 'dog'],
  'dog': ['dog', 'dog', 'dog', 'dog', 'mouse']
}
```
* So add2dict(d, k, v) does:
* If k (the key) is not in dictionary d, it adds it with an empty list.
* It appends the value v to the list at d[k].



In [12]:
for line in open(r"D:\Deepam\Projects\Text Classifier\robert_frost.txt"):
    tokens = remove_punctuation(line.rstrip().lower()).split()

    T = len(tokens)
    for i in range(T):
        t = tokens[i]
        if i == 0:
            # Measure the distribution of the first word
            initial[t] = initial.get(t, 0.) + 1
        else:
            t_1 = tokens[i-1]
            if i == T - 1:
                # measure probability of ending the line
                add2dict(second_order, (t_1, t), "END")
            if i == 1:
                # measure distribution of second word
                # give only first word
                add2dict(first_order, t_1, t)
            else:
                t_2 = tokens[i-2]
                add2dict(second_order, (t_2, t_1), t)

## 🧠 Goal of the Exercise

To build a language model that can generate new lines that sound like **Robert Frost** by learning the probabilities of word sequences from his existing lines. You're building:

| Component               | Description                                       |
|------------------------|---------------------------------------------------|
| Initial Distribution   | Which words typically begin a line?               |
| First-Order Transition | What's likely the second word, given the first?   |
| Second-Order Transition| What is the next word, given the previous two?    |

---

## 📦 Variables Involved

| Variable       | Purpose                                   | Example                                        |
|----------------|-------------------------------------------|------------------------------------------------|
| `initial`      | Counts how often a word starts a line     | `{'two': 1, 'and': 1}`                         |
| `first_order`  | Maps 1st word → possible 2nd words        | `{'two': ['roads'], 'and': ['sorry']}`        |
| `second_order` | Maps (word1, word2) → next word(s)        | `{('two', 'roads'): ['diverged']}`            |

---

## 📜 Code Breakdown

```python
for line in open("robert_frost.txt"):
    tokens = remove_punctuation(line.rstrip().lower()).split()
```

### 🔹 Step-by-Step:
| Step         | What It Does                                               |
|--------------|------------------------------------------------------------|
| Read Line    | Reads each line from the file                              |
| Clean Line   | Removes punctuation, converts to lowercase                 |
| Tokenize     | Splits line into a list of words (tokens)                  |

📌 **Example Line**:  
`"Two roads diverged in a yellow wood,"` →  
`tokens = ['two', 'roads', 'diverged', 'in', 'a', 'yellow', 'wood']`  
`T = 7`

---

## 🔁 Loop Over Each Token

```python
T = len(tokens)
for i in range(T):
    t = tokens[i]
```

Looping through each word in the line using its index `i`.

---

### ✅ Case 1: First Word (`i == 0`)

```python
if i == 0:
    initial[t] = initial.get(t, 0.) + 1
```

Adds count for the **first word** of the line.

📌 If the first word is `"two"` →  
`initial['two'] = 1`

Later you can normalize:
```python
P(w1) = count(w1) / total_lines
```

---

### ✅ Case 2: Second Word (`i == 1`)

```python
elif i == 1:
    t_1 = tokens[i - 1]
    add2dict(first_order, t_1, t)
```

For second word, link it to the first word.

📌 Example:
```python
tokens = ['two', 'roads', 'diverged']
i = 1 → t = 'roads', t_1 = 'two'
→ first_order = {'two': ['roads']}
```

---

### ✅ Case 3: Third Word & Beyond (`i >= 2`)

```python
else:
    t_2 = tokens[i - 2]
    t_1 = tokens[i - 1]
    add2dict(second_order, (t_2, t_1), t)
```

For 3rd+ word, consider previous **two** words.

📌 Example:
```python
tokens = ['two', 'roads', 'diverged', 'in']
i = 2 → t = 'diverged', t_1 = 'roads', t_2 = 'two'
→ second_order[('two', 'roads')] = ['diverged']
```

---

### ✅ Case 4: Last Word in Line (`i == T - 1`)

```python
if i == T - 1:
    add2dict(second_order, (t_1, t), "END")
```

Marks the **end of line** by appending `"END"` after the last two words.

📌 Example:
```python
tokens = ['two', 'roads', 'diverged', 'in', 'a', 'yellow', 'wood']
i = 6 → t = 'wood', t_1 = 'yellow'
→ second_order[('yellow', 'wood')] = ['END']
```

---

## 🔧 `add2dict()` Function

```python
def add2dict(d, k, v):
    if k not in d:
        d[k] = []
    d[k].append(v)
```

🔹 Ensures that if key `k` is not in dictionary `d`, it initializes it as a list.  
🔹 Appends value `v` to the list associated with key `k`.

---

## 🧪 Final Example of Dictionaries After One Line

If you process the line:

```
"Two roads diverged in a yellow wood"
```

Your dictionaries will look like:

```python
initial = {
    'two': 1
}

first_order = {
    'two': ['roads']
}

second_order = {
    ('two', 'roads'): ['diverged'],
    ('roads', 'diverged'): ['in'],
    ('diverged', 'in'): ['a'],
    ('in', 'a'): ['yellow'],
    ('a', 'yellow'): ['wood'],
    ('yellow', 'wood'): ['END']
}
```


In [14]:
# normalize the distributions
initial_total = sum(initial.values())
for t, c in initial.items():
    initial[t] = c / initial_total

In [15]:
# convert [cat, cat, cat, dog, dog, dog, dog, mouse, ...0
# into {cat: 0.5, dog:0.4, mouse: 0.1}

In [16]:
def list2pdict(ts):
    # turn each list of possibilities into a dictionary of probabilities
    d = {}
    n = len(ts)
    for t in ts:
        d[t] = d.get(t, 0.) + 1
    for t, c in d.items():
        d[t] = c/n
    return d

* ts = ['happy', 'sad', 'happy', 'tired', 'happy', 'sad']
* Step 1: Count frequency
```python
d = {
    'happy': 3,
    'sad': 2,
    'tired': 1
}
Step 2: Normalize
python
Copy code
total = 6

d = {
    'happy': 3 / 6 = 0.5,
    'sad':   2 / 6 ≈ 0.333,
    'tired': 1 / 6 ≈ 0.167
}
```

In [17]:
for t_1, ts in first_order.items():
    # replace list with dictionary of probabilities
    first_order[t_1] = list2pdict(ts)

In [18]:
for k, ts in second_order.items():
    second_order[k] = list2pdict(ts)

* Before:
```python
first_order = {
    'i': ['am', 'am', 'was'],
    'he': ['is', 'was'],
}

second_order = {
    ('i', 'am'): ['happy', 'tired', 'happy'],
    ('he', 'is'): ['smart', 'intelligent', 'smart', 'lazy']
}
```
* This means:
* After the word 'i', the next word could be 'am', 'am', or 'was'
* After 'i am', the next word could be 'happy', 'tired', or 'happy'

* These are just counts (raw lists). But we want to turn them into probability distributions — so the model can decide what to pick next based on probabilities, not just frequency.

First Order (before):
```python
'i': ['am', 'am', 'was']
# list2pdict → count: {'am': 2, 'was': 1}, total = 3
# → {'am': 2/3 ≈ 0.666, 'was': 1/3 ≈ 0.333}

'he': ['is', 'was']
# list2pdict → {'is': 0.5, 'was': 0.5}
🔹 After transformation:

first_order = {
    'i': {'am': 0.666, 'was': 0.333},
    'he': {'is': 0.5, 'was': 0.5}
}
🔹 Second Order (before):

('i', 'am'): ['happy', 'tired', 'happy']
# → {'happy': 0.666, 'tired': 0.333}

('he', 'is'): ['smart', 'intelligent', 'smart', 'lazy']
# → {'smart': 0.5, 'intelligent': 0.25, 'lazy': 0.25}
🔹 After transformation:

second_order = {
    ('i', 'am'): {'happy': 0.666, 'tired': 0.333},
    ('he', 'is'): {'smart': 0.5, 'intelligent': 0.25, 'lazy': 0.25}
}
```

In [19]:
def sample_word(d):
    # print 'd:', d
    p0 = np.random.random()
    # print "p0:", p0
    cumulative = 0
    for t, p in d.items():
        cumulative += p
        if p0 < cumulative:
            return t
    assert(False) # should never get here

🔍 Example:
```python
d = {'am': 0.6, 'was': 0.4}
Let’s say:

python
Copy code
p0 = 0.42
Now loop:

t	|p	   |cumulative	|Check p0 < cumulative?	|Action
'am'|	0.6|	0.6	    |0.42 < 0.6 ✅	        |return am"


So it will return 'am'.

Another case:

python
Copy code
p0 = 0.88
Loop:

t	 |p	   | cumulative| Check p0 < cumulative?|	Action
'am' |	0.6| 0.6	   |0.88 < 0.6 ❌	       |continue
'was'|	0.4| 1.0	   |0.88 < 1.0 ✅	       |return 'was'

🚨 What about this line?
```python
assert(False)  # should never get here
```
* This is a fail-safe. It means:
* If the loop runs completely and no word was returned, there’s a bug.
* It should never happen if all probabilities in d sum up to 1.0 (or close enough due to float precision).

In [20]:
def generate():
    for i in range(4): # generate 4 lines
        sentence = []

        # initial word
        w0 = sample_word(initial)
        sentence.append(w0)

        # sample second word
        w1 = sample_word(first_order[w0])
        sentence.append(w1)

        # second-order transitions until END
        while True:
            w2 = sample_word(second_order[(w0, w1)])
            if w2 == "END":
                break
            sentence.append(w2)
            w0 = w1
            w1 = w2
        print(' '.join(sentence))

In [23]:
generate()

i know
up to pass a winter eve
to make them out
and then someone


* Assume this is your trained model:
```python
initial = {'the': 0.6, 'i': 0.4}
first_order = {
  'the': {'dog': 1.0},
  'i': {'am': 1.0}
}
second_order = {
  ('the', 'dog'): {'barked': 0.5, 'ran': 0.5},
  ('dog', 'barked'): {'END': 1.0},
  ('dog', 'ran'): {'away': 1.0},
  ('ran', 'away'): {'END': 1.0},
  ('i', 'am'): {'sad': 1.0},
  ('am', 'sad'): {'END': 1.0}
}
```
* Calling **generate()** might output:
```css
the dog ran away
i am sad
the dog barked
i am sad
```

## Step-by-Step Implementation Plan
* If you want to rebuild this from scratch, follow this blueprint:
```python
🔹 Step 1: Load & Tokenize Text
# Example text
text = "the dog ran the dog barked"

# Tokenize
tokens = text.lower().split()
tokens.append("END")  # mark sentence end
🔹 Step 2: Build Raw Frequency Dictionaries
python
Copy code
initial = {}          # initial word counts
first_order = {}      # word -> [next words]
second_order = {}     # (word1, word2) -> [next words]

# Build counts
for i in range(len(tokens)-2):
    t0, t1, t2 = tokens[i], tokens[i+1], tokens[i+2]

    if i == 0:
        initial[t0] = initial.get(t0, 0) + 1

    if t0 not in first_order:
        first_order[t0] = []
    first_order[t0].append(t1)

    key = (t0, t1)
    if key not in second_order:
        second_order[key] = []
    second_order[key].append(t2)
    
🔹 Step 3: Normalize Distributions

# Normalize initial
total = sum(initial.values())
initial = {k: v / total for k, v in initial.items()}

# Helper to convert list -> probability dict
def list2pdict(lst):
    d = {}
    for word in lst:
        d[word] = d.get(word, 0) + 1
    total = len(lst)
    return {k: v / total for k, v in d.items()}

# Normalize first_order and second_order
first_order = {k: list2pdict(v) for k, v in first_order.items()}
second_order = {k: list2pdict(v) for k, v in second_order.items()}

🔹 Step 4: Define Sampling Function

import numpy as np

def sample_word(d):
    p0 = np.random.random()
    cumulative = 0
    for word, prob in d.items():
        cumulative += prob
        if p0 < cumulative:
            return word
    assert False, "Should not reach here if probabilities sum to 1"
    
🔹 Step 5: Generate Sentences
def generate():
    for _ in range(4):  # generate 4 lines
        sentence = []
        w0 = sample_word(initial)
        sentence.append(w0)

        w1 = sample_word(first_order[w0])
        sentence.append(w1)

        while True:
            w2 = sample_word(second_order[(w0, w1)])
            if w2 == "END":
                break
            sentence.append(w2)
            w0, w1 = w1, w2

        print(' '.join(sentence))
```

In [24]:
# Exercise:
#
# Determine the vocabulary size (V)
# We know that pi has shape V, A1 has shape V x V, and A2 has shape V x V x V
#
# In comparison, how many values are stored in our dictionaries?

In [25]:
# Exercise 2:
# We can skip the step where we accumulate all the possible next words in a list
# E.g. [cat, cat, dog, dog, dog, ...]

# Instead, like we do with initial state distribution, create the dictionary
# of counts directly as you loop through the data.
#
# You'll no longer need list2pdict()